## BRONZE TO SILVER LAYER

### Silver Layer - Stock Events History


In [1]:
# Importing Common Utility Function
import re

import pandas as pd

from StockETL import GlobalPath

In [2]:
# Import necessary libraries and utility functions
%run ../COMMON/common_utility.ipynb

USERNAME = 'ptprashanttripathi'


In [3]:
# Instantiate GlobalPath
stockdata_bronze_layer_path = GlobalPath("DATA/BRONZE/StockData")
stockevents_silver_file_path = GlobalPath(
    "DATA/SILVER/StockEvents/StockEvents_data.csv"
)
stockevents_silver_schema_file_path = GlobalPath(
    "CONFIG/DATA_CONTRACTS/SILVER/StockEvents.json"
)

### Data Processing

This section handles the collection of data from the Bronze layer.

- Files are checked for availability.
- Data is read from each file, processed, and stored in a list of DataFrames.
- Process stock event data from the Bronze layer and transform it for the Silver layer.


In [4]:
# Initialize an empty list to store DataFrames
df_stock_events_list = []

# Generate file paths for available CSV files in the Bronze layer
file_paths = check_files_availability(stockdata_bronze_layer_path, file_pattern="*.csv")

# Process each CSV file found in the Bronze layer
for file_path in file_paths:
    print(f"Processing file => {file_path}")

    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Extract the stock symbol by removing the date pattern from the file name
    df["symbol"] = re.sub(r"_\d{4}_\d{2}\.csv", "", file_path.name)

    # Append the DataFrame to the list
    df_stock_events_list.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(df_stock_events_list, ignore_index=True)

# Harmonize column names to ensure consistency
df = replace_punctuation_from_columns(df)

# Remove columns that contain only NA values
df.dropna(how="all", axis=1, inplace=True)

Number of Files Detected => 1700
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPITTECH_2024_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CTL_2024_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATACHEM_2022_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ADANIPORTS_2023_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/HAPPSTMNDS_2024_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPEL_2024_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CTL_2022_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/HAPPSTMNDS_2024_08.csv
Processing file => /home/runner/work/PortfolioTracker/Portfolio

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/URJA_2024_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CARTRADE_2024_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CTL_2023_01.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/SYNCOMF_2024_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ICICIB22_2021_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/BECTORFOOD_2021_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RTNPOWER_2022_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/SBI_LONG_TERM_EQUITY_FUND_DIRECT_PLAN_GROWTH_2025_01.csv
Processing file => /home/runner/work/PortfolioTracker/Portfo

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VOLTAS_2022_04.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPEL_2023_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VIJIFIN_2023_05.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATAMOTORS_2024_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/WIPRO_2024_06.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/PRAKASHSTL_2022_10.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/HAPPSTMNDS_2022_06.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/LLOYDSENGG_2024_05.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockDa

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CTL_2022_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CARTRADE_2021_10.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/PNB_2020_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ICICIB22_2020_05.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/WIPRO_2022_05.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/GREENPOWER_2023_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/GRAVITA_2022_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPEL_2022_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TRIDENT_202

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VIKASLIFE_2023_10.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CTL_2024_04.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RTNPOWER_2024_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPITTECH_2022_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CRESSAN_2023_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VISESHINFO_2022_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/SBICARD_2023_10.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TCS_2023_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATA

...

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/SBICARD_2022_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATAMOTORS_2022_10.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/BECTORFOOD_2024_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ZOMATO_2023_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/AWL_2023_11.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/HAPPSTMNDS_2023_01.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RTNPOWER_2022_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TRIDENT_2023_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockDat

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/HFCL_2023_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ADANIPORTS_2024_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/PNB_2020_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/YAMNINV_2022_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ADANIPORTS_2022_11.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RAGHUSYN_2022_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VIJIFIN_2024_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/YESBANK_2020_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TAT

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RTNINDIA_2024_11.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/BECTORFOOD_2021_04.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TRIDENT_2022_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPEL_2023_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/LLOYDSENGG_2022_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/IDEA_2020_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATACHEM_2022_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/WIPRO_2022_04.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RAG

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/SUZLON_2023_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/GREENPOWER_2022_04.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VISESHINFO_2022_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/WIPRO_2023_04.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/IRFC_2024_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATAPOWER_2021_04.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TRIDENT_2024_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPITTECH_2023_04.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/URJA_2023_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ZOMATO_2022_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/SBICARD_2024_06.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TTML_2023_11.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CRESSAN_2022_11.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/SUZLON_2022_05.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/IRFC_2024_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPITTECH_2025_01.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/WIPRO_2024_11

In [5]:
# Convert datetime strings to date objects for consistency
df["date"] = pd.to_datetime(df["date"]).dt.date

# Pivot the DataFrame: columns become rows
df = df.melt(
    id_vars=["date", "symbol"],
    value_vars=["dividends", "stock_splits", "capital_gains"],
    var_name="event",
    value_name="value",
)

# Convert event names to uppercase for uniformity
df["event"] = df["event"].str.upper()

# Round numerical values to 2 decimal places
df["value"] = df["value"].fillna(0).round(2)

# Filter out rows where the value is zero or less than a threshold
df = df[df["value"] > 0]

# Align Datafame with DataContract
df = align_with_datacontract(df, stockevents_silver_schema_file_path)

# Save the cleaned and transformed DataFrame as a CSV file in the Silver layer
df.to_csv(stockevents_silver_file_path, index=False)
print("SILVER Layer CSV file for Stock Events history successfully created at =>")
print(stockevents_silver_file_path)

# Log the DataFrame debug for verification
df.info()

DataContract loaded from => /home/runner/work/PortfolioTracker/PortfolioTracker/CONFIG/DATA_CONTRACTS/SILVER/StockEvents.json
SILVER Layer CSV file for Stock Events history successfully created at =>
/home/runner/work/PortfolioTracker/PortfolioTracker/DATA/SILVER/StockEvents/StockEvents_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    94 non-null     string 
 1   symbol  94 non-null     string 
 2   event   94 non-null     string 
 3   value   94 non-null     float64
dtypes: float64(1), string(3)
memory usage: 3.1 KB
